In [1]:
import os
import pickle
import numpy as np

In [12]:
imputation_method = 'MIDA'

params = {
    'COVID': {
        'n_imputations': 20,
        'n_runs': 200,
        'epochs': 50,
        'n_batches': 10,
        'hidden_layer_sizes': [128, 32],
        'alpha': 1.,
    },
    'MYOCARDIAL': {
        'n_imputations': 20,
        'n_runs': 200,
        'epochs': 40,
        'n_batches': 50,
        'hidden_layer_sizes': [64, 32],
        'alpha': 1.,
    },
    'NHANES': {
        'n_imputations': 20,
        'n_runs': 200,
        'epochs': 50,
        'n_batches': 50,
        'hidden_layer_sizes': [64, 32],
        'alpha': 1.,
    },
}

datasets = list(params.keys())

clean_datasets = {
    'COVID': 'COVI',
    'MYOCARDIAL': 'MYOC',
    'NHANES': 'NHAN',
}

approaches = [
    'SINGLE',
    'SINGLE-HOT-PATCH',
    'MULTIPLE',
    'HOT-PATCH',
]

clean_approaches = {
    'SINGLE': 'SI',
    'SINGLE-HOT-PATCH': 'S-HOT',
    'MULTIPLE': 'MI',
    'HOT-PATCH': 'M-HOT',
}

evaluation_metrics = [
    'bal_acc',
    'auc_ovo',
    'f1',
]

clean_metrics = {
    'bal_acc': 'bACC',
    'auc_ovo': 'AUC',
    'f1': 'F1',
}

num_lines = 1

In [13]:
print('\\hline')
line = 'Dataset & Metric & '
line += f'\\textit{{{clean_approaches["SINGLE"]}}} & '
line += f'\\textit{{{clean_approaches["SINGLE-HOT-PATCH"]}}} & '
line += f'\\textit{{{clean_approaches["MULTIPLE"]}}} & '
line += f'\\textit{{{clean_approaches["HOT-PATCH"]}}} \\\\'
print(line)
print('\\hline')

for dataset in datasets:
    n_imputations = params[dataset]['n_imputations']
    n_runs = params[dataset]['n_runs']
    epochs = params[dataset]['epochs']
    n_batches = params[dataset]['n_batches']
    hidden_layer_sizes = params[dataset]['hidden_layer_sizes']
    alpha = params[dataset]['alpha']
    exp_name = f'{dataset}_n{n_imputations}_r{n_runs}_e{epochs}_b{n_batches}' + \
               f'_i{"_".join(map(str, hidden_layer_sizes))}_a{str(alpha).replace(".", "_")}'
    if os.path.isfile(f'results/{exp_name}.pickle'):
        metrics = pickle.load(open(f'results/{exp_name}.pickle', 'rb'))
        line_means = f'{clean_datasets[dataset]} &'
        for i, metric in enumerate(evaluation_metrics):
            line_means = ''
            values = {}
            for m in evaluation_metrics:
                values[m] = {'S': [], 'M': []}
                for approach in approaches:
                    if metric == 'auc_ovo':
                        value = round(np.mean(metrics[imputation_method][approach][m]) / 100., 7)
                    else:
                        value = round(np.mean(metrics[imputation_method][approach][m]), 4)
                    values[m][clean_approaches[approach][0]].append(value)
            for a, approach in enumerate(approaches):
                if a == 0 and i == 0:
                    # Print dataset name
                    line_means += f'\\multirow{{{len(evaluation_metrics)}}}{{*}}{{{clean_datasets[dataset]}}} '
                if a == 0:
                    # Print metric
                    line_means += f'& {clean_metrics[metric]} '
                if metric == 'auc_ovo':
                    means = round(np.mean(metrics[imputation_method][approach][metric]) / 100., 7)
                else:
                    means = round(np.mean(metrics[imputation_method][approach][metric]), 4)
                bold = means == max(values[metric][clean_approaches[approach][0]])
                line_means += f'& $\\mathbf{{{means}}}$ ' if bold else f'& ${means}$ '
            print(line_means + '\\\\')
    print('\\hline')

\hline
Dataset & Metric & \textit{SI} & \textit{S-HOT} & \textit{MI} & \textit{M-HOT} \\
\hline
\multirow{3}{*}{COVI} & bACC & $88.1085$ & $\mathbf{88.2243}$ & $\mathbf{88.4101}$ & $88.3932$ \\
& AUC & $0.959175$ & $\mathbf{0.9597053}$ & $0.9606792$ & $\mathbf{0.9607641}$ \\
& F1 & $88.1598$ & $\mathbf{88.3403}$ & $\mathbf{88.473}$ & $88.4681$ \\
\hline
\multirow{3}{*}{MYOC} & bACC & $69.6988$ & $\mathbf{69.8614}$ & $70.0331$ & $\mathbf{70.2846}$ \\
& AUC & $0.8126983$ & $\mathbf{0.8143542}$ & $0.8419529$ & $\mathbf{0.8424803}$ \\
& F1 & $84.9747$ & $\mathbf{84.9862}$ & $85.6655$ & $\mathbf{85.7106}$ \\
\hline
\multirow{3}{*}{NHAN} & bACC & $63.4962$ & $\mathbf{63.5197}$ & $\mathbf{64.2441}$ & $64.1943$ \\
& AUC & $0.6982824$ & $\mathbf{0.6984358}$ & $\mathbf{0.7141273}$ & $0.7137177$ \\
& F1 & $63.2441$ & $\mathbf{63.2592}$ & $\mathbf{63.9746}$ & $63.9232$ \\
\hline
